In [ ]:
#Installing packages
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=bb3f35218810feb7236360f957b3e2c35137cf95551997682137d6f49aa74543
  Stored in directory: /tmp/pip-ephem-wheel-cache-puu84lwb/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
# Importing libraries
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np

import matplotlib.pyplot as plt
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('punkt')

import spacy 
from spacy import displacy

import random

# from rake_nltk import Rake

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

from gensim.scripts.glove2word2vec import glove2word2vec

# import en_core_web_lg
# nlp = en_core_web_lg.load()

from nltk.tokenize import word_tokenize

import gensim
from scipy import spatial

import csv
import tensorflow as tf

# import transformers
# transformers.logging.set_verbosity(transformers.logging.CRITICAL)

# from sentence_transformers import SentenceTransformer, util
# import nmslib

# distilbert = SentenceTransformer('distilbert-base-uncased')

import multiprocessing
import math

import time
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
import pandas as pd
import gensim
gensim.__version__

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'3.6.0'

In [ ]:
#Importing the cleaned dataset
movtag2 = pd.read_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/1_cleaned_tags.pkl")

In [ ]:
#extracting person, organisation, location and norp from the tags
%%time

person = []
org = []
gpe = []
norp = []
tags = []

for i in movtag2['combined_tags_nodup']:
  # random.shuffle(i)
  txt = ' '.join(i)
  doc = nlp(txt)

  entities = []
  labels = []


  for ent in doc.ents:
      entities.append(ent.text)
      labels.append(ent.label_)
      # position_start.append(ent.ORG)
      # position_end.append(ent.end_char)

      df_text = pd.DataFrame({'Entities':entities,'Labels':labels})
  
  lst_people = list(df_text[df_text.Labels == "PERSON"]['Entities'])
  lst_org = list(df_text[df_text.Labels == "ORG"]['Entities'])
  lst_gpe = list(df_text[(df_text.Labels == "GPE") | (df_text.Labels == "LOC")]['Entities'])
  lst_norp = list(df_text[df_text.Labels == "NORP"]['Entities'])

  lst_tags = [elem for elem in i if elem not in lst_people + lst_org + lst_gpe + lst_norp]

  person.append(lst_people)
  org.append(lst_org)
  gpe.append(lst_gpe)
  norp.append(lst_norp)
  tags.append(lst_tags)

movtag2['tag_people'] = person
movtag2['tag_org'] = org
movtag2['tag_norp'] = norp
movtag2['tag_location'] = gpe
movtag2['tags'] = tags

CPU times: user 12min 26s, sys: 3.52 s, total: 12min 29s
Wall time: 12min 31s


In [ ]:
movtag2.head(5)

,movieId,genres,combined_tags,genres_nodup,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]","[Owned, imdb top 250, Pixar, Pixar, time trave...","[Adventure, Animation, Children, Comedy, Fantasy]","[Owned, imdb, top, Pixar, time, travel, childr...","[chindren pixar, Tom Hanks, Woody, Tim Allen, ...","[imdb, Pixar, Disney, National Film Registry, ...",[],[],"[Owned, top, time, travel, children, comedy, f..."
1,2,"[Adventure, Children, Fantasy]","[Robin Williams, time travel, fantasy, based o...","[Adventure, Children, Fantasy]","[Robin, Williams, time, travel, fantasy, based...","[Robin Williams, Chris Van Allsburg, Kirsten D...",[Zathura],[],[],"[Robin, Williams, time, travel, fantasy, based..."
2,3,"[Comedy, Romance]","[funny, best friend, duringcreditsstinger, fis...","[Comedy, Romance]","[funny, best, friend, duringcreditsstinger, fi...","[Jack Lemmon, Walter Matthau grun, Howard Deut...",[comedinha de velhinhos engraÃƒÂ§ada old],[],[],"[funny, best, friend, duringcreditsstinger, fi..."
3,4,"[Comedy, Drama, Romance]","[based on novel or book, chick flick, divorce,...","[Comedy, Drama, Romance]","[based, novel, book, chick, flick, divorce, in...",[CLV],[],[],[],"[based, novel, book, chick, flick, divorce, in..."
4,5,[Comedy],"[aging, baby, confidence, contraception, daugh...",[Comedy],"[aging, baby, confidence, contraception, daugh...","[steve martin Comedy Touching CLV, Diane Keato...",[],[],[],"[aging, baby, confidence, contraception, daugh..."


In [ ]:
#cleaning the text further: removing words which are not present in the english dictionary
words = set(nltk.corpus.words.words())
clean_tags = []
for k in movtag2['tags']:
  lst = []
  for i in [w for w in k]:
    j = i.split(" ")
    lst.append([l for l in j if l in words])

  lst = [' '.join(w) for w in lst]
  lst = [w for w in lst if w != '']

  clean_tags.append(lst)

movtag2['tags'] = clean_tags

In [ ]:
movtag2.drop(['genres', 'combined_tags', 'genres_nodup'], axis=1, inplace = True)

In [ ]:
movtag2.head(2)

,movieId,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags
0,1,"[Owned, imdb, top, Pixar, time, travel, childr...","[chindren pixar, Tom Hanks, Woody, Tim Allen, ...","[imdb, Pixar, Disney, National Film Registry, ...",[],[],"[top, time, travel, comedy, funny, witty, anim..."
1,2,"[Robin, Williams, time, travel, fantasy, based...","[Robin Williams, Chris Van Allsburg, Kirsten D...",[Zathura],[],[],"[Robin, time, travel, fantasy, based, book, bo..."


In [ ]:
movtag2.to_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/2_ner_tags.pkl")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/2_ner_tags.pkl")

In [8]:
data[9:10]

,movieId,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags
9,10,"[Bond, boys, toys, gadgets, secret, service, s...","[James Bond, Pierce Brosnan, Sean Bean, Famke ...","[kgb, Brosnon Bonds, BD, EMP, Tumey, MTSKAF]",[],"[petersburg, russia, Caribbean, Latin America,...","[secret, service, sequel, violence, computer, ..."
